# Imports

In [ ]:
N_threads = 6

In [ ]:
%load_ext autoreload

import sys, os
os.environ["OMP_NUM_THREADS"] = str(N_threads)
os.environ["OPENBLAS_NUM_THREADS"] = str(N_threads)
os.environ["MKL_NUM_THREADS"] = str(N_threads)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(N_threads)
os.environ["NUMEXPR_NUM_THREADS"] = str(N_threads)

import numpy as np
import sklearn as skl

import SBIBE as sbibe

%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
plt.close('all')

np.random.seed(seed=0)

# Extract train

In [ ]:
dict_bounds = {
    'omega_cold'    :  [0.23, 0.4],
    'omega_baryon'  :  [0.04, 0.06],
    'hubble'        :  [0.6, 0.8],
    'ns'            :  [0.92, 1.01],
    'sigma8_cold'   :  [0.73, 0.9]
}
theta_train = sbibe.sbi_data_utils.sample_latin_hypercube(dict_bounds, N_points=3000, seed=0)
xx_train, kk = sbibe.sbi_data_utils.get_xx(dict_bounds, theta_train)

In [ ]:
print(theta_train.shape)
print(xx_train.shape)

# Train the network

In [ ]:
# scaler = skl.preprocessing.StandardScaler()
# norm_xx_train = scaler.fit_transform(xx_train)

inference, posterior =sbibe.sbi_utils.train_model(
    theta_train,
    xx_train,
    prior=sbibe.sbi_utils.get_prior(dict_bounds)
)

# Extract test - posterior inference - rank stats

In [ ]:
# ------------------ extract test ------------------ #

dict_bounds_test = {
    'omega_cold'    :  [0.25, 0.38],
    'omega_baryon'  :  [0.042, 0.058],
    'hubble'        :  [0.62, 0.78],
    'ns'            :  [0.93, 1.00],
    'sigma8_cold'   :  [0.75, 0.88],
}
theta_test = sbibe.sbi_data_utils.sample_latin_hypercube(dict_bounds_test, N_points=987, seed=0)
xx_test, kk = sbibe.sbi_data_utils.get_xx(dict_bounds_test, theta_test)

# ------------------ posterior inference ------------------ #

# norm_xx_test = scaler.transform(xx_test)

inferred_theta_test = sbibe.sbi_utils.sample_posteriors_theta_test(
    posterior,
    xx_test,
    dict_bounds_test
)

# ------------------ rank stats ------------------ #

ranks = sbibe.sbi_utils.compute_ranks(theta_test, inferred_theta_test)

# Visualizations

In [ ]:
custom_titles = [
    r'$\Omega_\mathrm{c}$',
    r'$\Omega_\mathrm{b}$',
    r'$h$',
    r'$n_\mathrm{s}$',
    r'$\sigma_{8,\mathrm{c}}$'
]

N_examples = 5
indexes = np.random.choice(inferred_theta_test.shape[0], N_examples, replace=False)

## Visualize train and test examples

In [ ]:
custom_lines = [
    mpl.lines.Line2D([0], [0], color='royalblue', ls='-', lw=1, marker=None, markersize=9),
    mpl.lines.Line2D([0], [0], color='k', ls='-', lw=1, marker=None, markersize=9)
]

fig, ax = sbibe.plot_utils.simple_plot(
    x_label=r'$\log_{10}\mathrm{Wavenumber}\, k \left[ h\, \mathrm{Mpc}^{-1} \right]$',
    y_label=r'$\log_{10}P(k) \left[ \left(h^{-1} \mathrm{Mpc}\right)^{3} \right]$',
    custom_labels=[r'Train', r'Val'],
    custom_lines=custom_lines
)

tmp_xx_plot = xx_train
tmp_xx_plot = tmp_xx_plot[np.random.choice(tmp_xx_plot.shape[0], tmp_xx_plot.shape[0], replace=False)].T
ax.plot(np.log10(kk), tmp_xx_plot, c='royalblue', alpha=0.2, lw=0.5)

tmp_xx_plot = xx_test
tmp_xx_plot = tmp_xx_plot[np.random.choice(tmp_xx_plot.shape[0], tmp_xx_plot.shape[0], replace=False)].T
ax.plot(np.log10(kk), tmp_xx_plot, c='k', alpha=0.1, lw=0.5)

plt.tight_layout()
plt.show()

## (Optional) Visualize xx associatted with inferred posteriors

In [ ]:
tmp_inferred_xx_test = sbibe.sbi_data_utils.compute_baccoemu_predictions_batch(
    inferred_theta_test[indexes],
    list(dict_bounds.keys())
)

fig, ax, ax_res = sbibe.plot_utils.plot_xx_from_sampled_posteriors(xx_test[indexes], tmp_inferred_xx_test, kk)
plt.show()

## Posteriors examples

In [ ]:
colors = sbibe.plot_utils.get_N_colors(N_examples, mpl.colormaps['prism'])
for ii_sample in range(N_examples):
    fig, axs = sbibe.plot_utils.corner_plot(
        theta_test[ii_sample],
        inferred_theta_test[ii_sample],
        custom_titles,
        dict_bounds,
        color_infer=colors[ii_sample]
    )
    plt.show()

## Rank statistics

In [ ]:
fig, axs = sbibe.plot_utils.plot_rank_statistcis(ranks, inferred_theta_test.shape[1], custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.05)
plt.show()

## Inference errors

In [ ]:
fig, axs = sbibe.plot_utils.plot_parameter_prediction_vs_truth(inferred_theta_test, theta_test, custom_titles)
plt.tight_layout()
plt.subplots_adjust(wspace=0.6)
plt.show()